# Generating SEDs for a semi-analytic model

In [1]:
import numpy as np
import torch
import starduster

In this tutorial, we generate some multi-wavelength SEDs for a semi-analytic model. We assume that the semi-analytic model can provide the following properties. For this example, we sample these properties randomly.

In [2]:
rstate = np.random.RandomState(111)

n_gal = 5
# Inlincation angel [deg]
theta = rstate.uniform(0, 90, n_gal)
# Dust mass [M_sol]
m_dust = 10**rstate.uniform(6, 8, n_gal)
# Dust disk radius [kpc]
r_dust = 10**rstate.uniform(-2, 2.5, n_gal) 
# Stellar disk radius [kpc]
r_disk = 10**rstate.uniform(-2, 2.5, n_gal)
# Bulge radius [kpc]
r_bulge = 10**rstate.uniform(-0.5, 1.5, n_gal)

We also need the star formation history of the stellar disk and bulge, which should have the following form.

In [3]:
# Array size for the star formation history
n_sfh = 10
# Stellar age bins [yr]
age_bins = np.logspace(5.8, 10.5, n_sfh + 1)
# Mass in each stellar age bin [M_sol]
sfh_mass_disk = 10**rstate.uniform(2, 8, [n_gal, n_sfh])
sfh_mass_bulge = 10**rstate.uniform(2, 8, [n_gal, n_sfh])
# Metal mass in each stellar age bin [M_sol]
sfh_metal_mass_disk = 10**rstate.uniform(-2, 4, [n_gal, n_sfh])
sfh_metal_mass_bulge = 10**rstate.uniform(-2, 4, [n_gal, n_sfh])

In [4]:
sed_model = starduster.MultiwavelengthSED.from_builtin()
converter = starduster.SemiAnalyticConventer(sed_model, age_bins)

We can then use ``SemiAnalyticConventer`` to convert the properties to format that can be accepted SED model. The code distributes the mass into the built-in age bins according the time fraction, and linearly intepolates the metallicity to the grid.

In [5]:
sed_model = starduster.MultiwavelengthSED.from_builtin()
converter = starduster.SemiAnalyticConventer(sed_model, age_bins)
gp, sfh_disk, sfh_bulge = converter(
    theta=theta,
    m_dust=m_dust,
    r_dust=r_dust,
    r_disk=r_disk,
    r_bulge=r_bulge,
    sfh_mass_disk=sfh_mass_disk,
    sfh_metal_mass_disk=sfh_metal_mass_disk,
    sfh_mass_bulge=sfh_mass_bulge,
    sfh_metal_mass_bulge=sfh_metal_mass_bulge
)
with torch.no_grad():
    spectra = sed_model(gp, sfh_disk, sfh_bulge)